## Data


In order to create the required prediction model, we need to find the dependence of the severity of the accident on parameters that can be collected in real time, such as weather conditions.

First, we need to use statistics about road traffic accidents to determine the factors that influence their severity. For these purposes, data from SDOT Traffic Management Division were used. 

All collisions provided by SPD and recorded by Traffic Records with weekly update frequency. This includes all types of collisions in timeframe from 2004 to present. This dataset includes many attributes that describe all the circumstances of the accident, the number of victims and their severity.


### Data acqusition

At this stage, we need to upload a .csv file from the Internet source to our Python environment and create a dataframe using pandas to perform data analysis and derive some additional info from our raw data to define, which attributes could be potentially useful for future prediction model. 

In [121]:
import pandas as pd
data_path = "https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv"
df = pd.read_csv(data_path, low_memory=False)

After reading the dataset, it is necessary to look at the data frame to get a better intuition. Let's print the first 10 rows of our dataset.

In [122]:
df.head(10)

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N
5,1,-122.387598,47.690575,6,320840,322340,E919477,Matched,Intersection,36974.0,...,Dry,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
6,1,-122.338485,47.618534,7,83300,83300,3282542,Matched,Intersection,29510.0,...,Wet,Daylight,NaN,8344002.0,NaN,10,Entering at angle,0,0,N
7,2,-122.320780,47.614076,9,330897,332397,EA30304,Matched,Intersection,29745.0,...,Dry,Daylight,NaN,NaN,NaN,5,Vehicle Strikes Pedalcyclist,6855,0,N
8,1,-122.335930,47.611904,10,63400,63400,2071243,Matched,Block,NaN,...,Dry,Daylight,NaN,6166014.0,NaN,32,One parked--one moving,0,0,N
9,2,-122.384700,47.528475,12,58600,58600,2072105,Matched,Intersection,34679.0,...,Dry,Daylight,NaN,6079001.0,NaN,10,Entering at angle,0,0,N


As we can see the names of the attributes in our dataset are written using abbreviations. To decrypt them, in addition to the dataset, it is also necessary to download a file with metadata using a following link https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Metadata.pdf, which contains descriptions of the attributes. This will allow us to understand what information we have.

### Data understanding

#### Defining a target function

First of all, we must determine which of these attributes we will predict as a target function in our model. Our task is to predict the severity of a road traffic accident, so we will use SEVERITYCODE as a target function. It corresponds to the severity of the collision and is a discrete value. Therefore, we will use classification model for prediction. 



#### Selecting the required attributes

Based on Metadata, we can preliminary analyze the attributes and filter out those that will not be useful for creating a prediction model. 

At first, we can delete the specific codes and definitions that SDOT uses for its reports, since this information cannot in any way be used for data analysis. This includes the following attributes:

* INCKEY - A unique key for the incident
* COLDETKEY - Secondary key for the incident
* SDOT_COLCODE - A code given to the collision by SDOT
* SDOT_COLDESC - A description of the collision corresponding to the collision code
* SDOTCOLNUM - A number given to the collision by SDOT
* REPORTNO - A number of report
* STATUS
* EXCEPTRSNCODE
* EXCEPTRSNDESC



In [123]:
df.drop(['INCKEY', 'COLDETKEY', 'SDOT_COLCODE', 'SDOT_COLDESC', 'SDOTCOLNUM', 'REPORTNO', 'STATUS', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC'], axis=1, inplace=True)

Since for our task it is necessary that the algorithm be universal and can be applied in different regions, we need to weed out those attributes that are relevant only for Seattle. This includes all attributes that contain specific location data:

* OBJECTID - ESRI unique identifier
* SHAPE - ESRI geometry field
* INTKEY - Key that corresponds to the intersection associated with a collision
* LOCATION - Description of the general location of the collision
* SEGLANEKEY - A key for the lane segment in which the collision occurred
* CROSSWALKKEY - A key for the crosswalk at which the collision occurred
* ADDRTYPE - Collision address type

In [124]:
df.drop(['OBJECTID', 'X', 'Y', 'INTKEY', 'LOCATION', 'SEGLANEKEY', 'CROSSWALKKEY', 'ADDRTYPE'], axis=1, inplace=True)

This report contains attributes that contain information about the number of people and vehicles involved in the incident, as well as information about the type of incident. Although they give a more complete understanding of the scale of the incident, they cannot be used as input parameters for the model, since this data was obtained after the incident.

* COLLISIONTYPE - Collision type
* PERSONCOUNT - The total number of people involved in the collision
* PEDCOUNT - The number of pedestrians involved in the collision. 
* PEDCYLCOUNT - The number of bicycles involved in the collision. 
* VEHCOUNT - The number of vehicles involved in the collision
* ST_COLCODE - A code provided by the state that describes the collision
* ST_COLDESC - A description that corresponds to the state’s coding designation 
* HITPARKEDCAR - Whether or not the collision involved hitting a parked car
* JUNCTIONTYPE - Category of junction at which collision took place 
* PEDROWNOTGRNT - Whether or not the pedestrian right of way was not granted

In [125]:
df.drop(['COLLISIONTYPE', 'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'ST_COLCODE', 'ST_COLDESC', 'HITPARKEDCAR', 'JUNCTIONTYPE', 'PEDROWNOTGRNT'], axis=1, inplace=True)

We delete attributes containing date and time stamps of incidents, because based on this information, we cannot make assumptions about a possible accident in the future and its severity:

* INCDATE - The date of the incident
* INCDTTM - The date and time of the incident

In [126]:
df.drop(['INCDATE', 'INCDTTM'], axis=1, inplace=True)

This dataset has two SEVERITYCODE columns that duplicate each other. Therefore, we need to remove one of them.

In [127]:
df.drop(['SEVERITYCODE.1'], axis=1, inplace=True)
df.head(10)

,SEVERITYCODE,SEVERITYDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,SPEEDING
0,2,Injury Collision,NaN,N,Overcast,Wet,Daylight,NaN
1,1,Property Damage Only Collision,NaN,0,Raining,Wet,Dark - Street Lights On,NaN
2,1,Property Damage Only Collision,NaN,0,Overcast,Dry,Daylight,NaN
3,1,Property Damage Only Collision,NaN,N,Clear,Dry,Daylight,NaN
4,2,Injury Collision,NaN,0,Raining,Wet,Daylight,NaN
5,1,Property Damage Only Collision,NaN,N,Clear,Dry,Daylight,NaN
6,1,Property Damage Only Collision,NaN,0,Raining,Wet,Daylight,NaN
7,2,Injury Collision,NaN,N,Clear,Dry,Daylight,NaN
8,1,Property Damage Only Collision,NaN,0,Clear,Dry,Daylight,NaN
9,2,Injury Collision,NaN,0,Clear,Dry,Daylight,NaN


The factors that led to the accident, such as the driver being under the influence of alcohol or drugs, speeding or inattention, affect the severity of the accident. But unfortunately these factors cannot be predicted in advance when applying a route for our users and they cannot be used for our model. Therefore, it is necessary to remove all cases of accidents that were caused by one of these factors, so as not to distort our statistics.

In [128]:
df['SPEEDING'].unique()
indexSPEEDING = df[ df['SPEEDING'] == 'Y' ].index
df.drop(indexSPEEDING , inplace=True)

In [129]:
df['INATTENTIONIND'].unique()
indexINATTENTIONIND = df[ df['INATTENTIONIND'] == 'Y' ].index
df.drop(indexINATTENTIONIND , inplace=True)

In [130]:
df['UNDERINFL'].unique()
indexUNDERINFL1 = df[ df['UNDERINFL'] == '1' ].index
indexUNDERINFL2 = df[ df['UNDERINFL'] == 'Y' ].index
df.drop(indexUNDERINFL1 , inplace=True)
df.drop(indexUNDERINFL2 , inplace=True)
df.head(100)

,SEVERITYCODE,SEVERITYDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,SPEEDING
0,2,Injury Collision,NaN,N,Overcast,Wet,Daylight,NaN
1,1,Property Damage Only Collision,NaN,0,Raining,Wet,Dark - Street Lights On,NaN
2,1,Property Damage Only Collision,NaN,0,Overcast,Dry,Daylight,NaN
3,1,Property Damage Only Collision,NaN,N,Clear,Dry,Daylight,NaN
4,2,Injury Collision,NaN,0,Raining,Wet,Daylight,NaN
5,1,Property Damage Only Collision,NaN,N,Clear,Dry,Daylight,NaN
6,1,Property Damage Only Collision,NaN,0,Raining,Wet,Daylight,NaN
7,2,Injury Collision,NaN,N,Clear,Dry,Daylight,NaN
8,1,Property Damage Only Collision,NaN,0,Clear,Dry,Daylight,NaN
9,2,Injury Collision,NaN,0,Clear,Dry,Daylight,NaN


After all cases caused by these factors have been deleted, we can delete the corresponding columns.

In [131]:
df.drop(['INATTENTIONIND', 'UNDERINFL', 'SPEEDING'], axis=1, inplace=True)
df.head(10)

,SEVERITYCODE,SEVERITYDESC,WEATHER,ROADCOND,LIGHTCOND
0,2,Injury Collision,Overcast,Wet,Daylight
1,1,Property Damage Only Collision,Raining,Wet,Dark - Street Lights On
2,1,Property Damage Only Collision,Overcast,Dry,Daylight
3,1,Property Damage Only Collision,Clear,Dry,Daylight
4,2,Injury Collision,Raining,Wet,Daylight
5,1,Property Damage Only Collision,Clear,Dry,Daylight
6,1,Property Damage Only Collision,Raining,Wet,Daylight
7,2,Injury Collision,Clear,Dry,Daylight
8,1,Property Damage Only Collision,Clear,Dry,Daylight
9,2,Injury Collision,Clear,Dry,Daylight


As a result, only those parameters remained that could potentially affect the severity of the accident and which can be collected in real time in order to always give the user an up-to-date prediction. These include:

* WEATHER -  weather conditions during the time of the collision 
* LIGHTCOND - The light conditions during the collision
* ROADCOND - The condition of the road during the collision


### Data pre-processing


#### Dealing with missing values 

Let's check if the selected parameters contain missing values or if the selected parameter are unknown. If so then such cases should be removed from the dataset.

In [132]:
df['WEATHER'].unique()
indexWEATHER = df[ df['WEATHER'] == 'Unknown' ].index
df.drop(indexWEATHER , inplace=True)


In [133]:
df['ROADCOND'].unique()
indexROADCOND = df[ df['ROADCOND'] == 'Unknown' ].index
df.drop(indexROADCOND , inplace=True)


In [134]:
df['LIGHTCOND'].unique()
indexLIGHTCOND = df[ df['LIGHTCOND'] == 'Unknown' ].index
df.drop(indexLIGHTCOND , inplace=True)

In [135]:
df.dropna(subset=['WEATHER', 'ROADCOND', 'LIGHTCOND'])

,SEVERITYCODE,SEVERITYDESC,WEATHER,ROADCOND,LIGHTCOND
0,2,Injury Collision,Overcast,Wet,Daylight
1,1,Property Damage Only Collision,Raining,Wet,Dark - Street Lights On
2,1,Property Damage Only Collision,Overcast,Dry,Daylight
3,1,Property Damage Only Collision,Clear,Dry,Daylight
4,2,Injury Collision,Raining,Wet,Daylight
5,1,Property Damage Only Collision,Clear,Dry,Daylight
6,1,Property Damage Only Collision,Raining,Wet,Daylight
7,2,Injury Collision,Clear,Dry,Daylight
8,1,Property Damage Only Collision,Clear,Dry,Daylight
9,2,Injury Collision,Clear,Dry,Daylight


#### Exploratory data analysis

In [136]:
df['SEVERITYCODE'].value_counts().to_frame()

,SEVERITYCODE
1,90851
2,40925


In [137]:
df['WEATHER'].value_counts().to_frame()

,WEATHER
Clear,81801
Raining,23598
Overcast,20022
Snowing,588
Fog/Smog/Smoke,378
Other,223
Sleet/Hail/Freezing Rain,80
Blowing Sand/Dirt,39
Severe Crosswind,20
Partly Cloudy,5


In [138]:
df['ROADCOND'].value_counts().to_frame()

,ROADCOND
Dry,91784
Wet,33538
Ice,675
Snow/Slush,566
Other,74
Oil,56
Standing Water,52
Sand/Mud/Dirt,50


In [139]:
df['LIGHTCOND'].value_counts().to_frame()

,LIGHTCOND
Daylight,86498
Dark - Street Lights On,32092
Dusk,4374
Dawn,1794
Dark - No Street Lights,1011
Dark - Street Lights Off,768
Other,162
Dark - Unknown Lighting,9


In [140]:
df_test = df[['LIGHTCOND', 'WEATHER', 'ROADCOND', 'SEVERITYCODE']]

In [141]:
df_grp = df_test.groupby (['LIGHTCOND', 'WEATHER', 'ROADCOND' ], as_index = False).mean()
df_sort = df_grp.sort_values('SEVERITYCODE', ascending=False)
df_sort

,LIGHTCOND,WEATHER,ROADCOND,SEVERITYCODE
19,Dark - No Street Lights,Raining,Ice,2.000000
88,Dark - Street Lights On,Sleet/Hail/Freezing Rain,Dry,2.000000
183,Dusk,Clear,Oil,2.000000
184,Dusk,Clear,Other,2.000000
24,Dark - No Street Lights,Sleet/Hail/Freezing Rain,Wet,2.000000
22,Dark - No Street Lights,Raining,Standing Water,2.000000
190,Dusk,Other,Ice,2.000000
170,Daylight,Sleet/Hail/Freezing Rain,Dry,2.000000
17,Dark - No Street Lights,Partly Cloudy,Dry,2.000000
195,Dusk,Overcast,Other,2.000000


From this table we can see the most dangerous and safest combinations of attributes. 

#### Turning categorical variables into quantitative variables

Most statistical models cannot take in objects or strings as input and for model training only take the numbers as inputs. In our dataset all input values are categorical values. For further analysis, we have to convert these variables into some form of numeric format. 

In [142]:
dummy_variable_1 = pd.get_dummies(df["WEATHER"])
dummy_variable_1.head()

,Blowing Sand/Dirt,Clear,Fog/Smog/Smoke,Other,Overcast,Partly Cloudy,Raining,Severe Crosswind,Sleet/Hail/Freezing Rain,Snowing
0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0


In [143]:
df = pd.concat([df, dummy_variable_1], axis=1)

In [144]:
df.drop("WEATHER", axis = 1, inplace=True)

In [145]:
dummy_variable_2 = pd.get_dummies(df["ROADCOND"])
dummy_variable_2.head()

,Dry,Ice,Oil,Other,Sand/Mud/Dirt,Snow/Slush,Standing Water,Wet
0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1


In [146]:
df = pd.concat([df, dummy_variable_2], axis=1)

In [147]:
df.drop("ROADCOND", axis = 1, inplace=True)

In [148]:
dummy_variable_3 = pd.get_dummies(df["LIGHTCOND"])
dummy_variable_3.head()

,Dark - No Street Lights,Dark - Street Lights Off,Dark - Street Lights On,Dark - Unknown Lighting,Dawn,Daylight,Dusk,Other
0,0,0,0,0,0,1,0,0
1,0,0,1,0,0,0,0,0
2,0,0,0,0,0,1,0,0
3,0,0,0,0,0,1,0,0
4,0,0,0,0,0,1,0,0


In [149]:
df = pd.concat([df, dummy_variable_3], axis=1)

In [150]:
df.drop("LIGHTCOND", axis = 1, inplace=True)

In [151]:
df.head(10)

,SEVERITYCODE,SEVERITYDESC,Blowing Sand/Dirt,Clear,Fog/Smog/Smoke,Other,Overcast,Partly Cloudy,Raining,Severe Crosswind,...,Standing Water,Wet,Dark - No Street Lights,Dark - Street Lights Off,Dark - Street Lights On,Dark - Unknown Lighting,Dawn,Daylight,Dusk,Other
0,2,Injury Collision,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1,1,Property Damage Only Collision,0,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,0
2,1,Property Damage Only Collision,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,Property Damage Only Collision,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,2,Injury Collision,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
5,1,Property Damage Only Collision,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,1,Property Damage Only Collision,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
7,2,Injury Collision,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8,1,Property Damage Only Collision,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,2,Injury Collision,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


After one-hot-encoding is our dataset ready to be used for machine learning algorithms.  